In [1]:
import pandas as pd
import numpy as np

In [2]:
def load_data(path):
    user = pd.read_csv(path + 'user.csv',header=None)
    item = pd.read_csv(path + 'item.csv',header=None)
    data = pd.read_csv(path + 'user_behavior.csv',header=None)

    data.columns = ['userID','itemID','behavior','timestamp']
    data['day'] = data['timestamp'] // 86400
    data['hour'] = data['timestamp'] // 3600 % 24

    ## 生成behavior的加权
    data['day_hour'] = data['day'] + data['hour'] / float(24)
    data.loc[data['behavior']=='pv','behavior'] = 1
    data.loc[data['behavior']=='fav','behavior'] = 2
    data.loc[data['behavior']=='cart','behavior'] = 3
    data.loc[data['behavior']=='buy','behavior'] = 1
    max_day = max(data['day'])
    min_day = min(data['day'])
    data['behavior'] = (1 - (max_day-data['day_hour']+2)/(max_day-min_day+2)) * data['behavior'] 

    item.columns = ['itemID','category','shop','brand']
    user.columns = ['userID','sex','age','ability']

    data = pd.merge(left=data, right=item, on='itemID',how='left')
    data = pd.merge(left=data, right=user, on='userID',how='left')

    return user, item, data
    
def get_unique_inorder(x, k=50):
    result = []
    flag = set()
    for i in x:
        if i[0] not in flag:
            result.append(i)
            flag.add(i[0])
        if len(flag) > k:
            break
    return result

def get_recall_list(train, targetDay, k=300):
    train_logs = dict()
    
    if targetDay > max(train['day']):
        for row in train[['userID','itemID','behavior']].values:
            train_logs.setdefault(row[0], dict())
            if row[1] in upward_map:
                train_logs[row[0]].setdefault(upward_map[row[1]],0)
                train_logs[row[0]][upward_map[row[1]]] = max(train_logs[row[0]][upward_map[row[1]]],row[2])
    else:
        user_List_test = set(train.loc[train['day']==targetDay,'userID'])
        train = train[train['day'] < targetDay]
        
        for row in train[['userID','itemID','behavior']].values:
            if row[0] in user_List_test:
                train_logs.setdefault(row[0], dict())
                if row[1] in upward_map:
                    train_logs[row[0]].setdefault(upward_map[row[1]],0)
                    train_logs[row[0]][upward_map[row[1]]] = max(train_logs[row[0]][upward_map[row[1]]],row[2])

    for each_user in train_logs:
        sum_value = sum(train_logs[each_user].values())
        if sum_value > 0:
            for each_item in train_logs[each_user]:
                train_logs[each_user][each_item] /= sum_value            

    result_logs = dict()    
    for u in train_logs:
        result_logs.setdefault(u, list())
        for i in set(train_logs[u].keys()):
            if i in item_dict:
                tmp_list = [ (x[0], train_logs[u][i]*x[1]) for x in item_dict[i]]
                result_logs[u] += tmp_list
            
    for u in result_logs:
        result_logs[u] = get_unique_inorder([(downward_map[x[0]], x[1]) for x in sorted(result_logs[u], key=lambda x:x[1], reverse=True)
                          if x[0] not in train_logs[u]], k=300)  
    
    return result_logs


def generate_pairs(recall):
    result = []
    for u in recall:
        for i in recall[u]:
            result.append([u,i[0],i[1]])
    return result

def reshape_recall_to_dataframe(recall):
    result = generate_pairs(recall)
    result = pd.DataFrame(result)
    result.columns = ['userID','itemID','apriori']
    return result


In [ ]:
#path = './'
path = '../ECommAI_EUIR_round2_train_20190816/'

## The target date(16 means online, 15 means underline test, 14 means underline train)
targetday = 15

## The lenth of recall list, the default is 300
lenth = 300

## The name of generated recall file

name = 'recall_list_round2_%dday_%dlenth.csv'%(targetday, lenth)


user, item, data = load_data(path = path)   

#tempory_path = './tempory_file/'
tempory_path = './'
f = open(tempory_path + 'upward_map.txt','r')

upward_map = f.read()
upward_map = eval(upward_map)
f.close()
    
f = open(tempory_path + 'downward_map.txt','r')
downward_map = f.read()
downward_map = eval(downward_map)
f.close()

f = open(tempory_path + 'item_Apriori.txt','r')
tmp = f.read()
item_dict = eval(tmp)
f.close()

注意，下面这一行代码是在256GB内存的机器上运行的。如果你的机器配置不够256GB，为你提供两种解决方案：
1. 使用最近日期的数据运行，依然会占用较大内存，对结果有一定影响：

          data = data[data['day'] >= K] ##这里K设置为12/13等靠后的天数
          
2. 【推荐】在Step 6_Sta_for_SparseMatrix.ipynb 中把下面代码中的参数500调小为300，对结果影响较小，内存需求很小：

          item_dict[i] = sorted(tmp_list,key=lambda x:x[1], reverse=True)[:500]
          
当然这部分是完全可以在不减少精度的情况下批处理优化的，但是由于当时比赛时间不够没有优化，而且有大内存机器。

In [4]:
recall_logs = get_recall_list(data, targetDay=targetday, k=lenth)

In [5]:
recall_df = reshape_recall_to_dataframe(recall_logs)

In [6]:
temp = pd.merge(left=recall_df, right=data[data['day'] == targetday][['userID','itemID','behavior']], 
         on=['userID','itemID'], how='left').rename(columns={'behavior':'label'})

In [ ]:
len(set(recall_df['userID']) & set(data[data['day'] == targetday]['userID']))

In [ ]:
len(set(recall_df['userID']))

In [ ]:
recall_df.to_csv(name, index=False)